In [1]:
#modules
import pandas as pd
import numpy as np
import Quandl as q
import pickle
import datetime
import matplotlib.pyplot as plt

# defining style
from matplotlib import style
style.use("fivethirtyeight")

#Quandl API key
api_key = "ZeTUAsZV7Vz4HFa7jsFk"

In [2]:
#extracting the state abbreviations from wiki
def state_list():
    all_state = pd.read_html("https://simple.wikipedia.org/wiki/List_of_U.S._states")
    #print(all_state[0][0][1:])
    return all_state[0][0][1:]

#state_list()

In [19]:
#function to get data from Quandl using the API
def get_quandl_data():
    all_states = state_list() #storing the states
    main_df = pd.DataFrame() #defining empty dataframe
    
    for each_state in all_states:
        query = "FMAC/HPI_"+str(each_state)
        df = q.get(query, authtoken = api_key)
        df[each_state] = (df[each_state]-df[each_state][0])/df[each_state][0] * 100.0
        
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df)
            
    #print(main_df[[0]])
    main_df.to_pickle("state_data.pickle")
    
get_quandl_data()



In [20]:
#print(state_data_df["AL"][0])
#print(state_data_df.AL[0])
#x =(state_data_df["AL"]-state_data_df["AL"][0]) / state_data_df["AL"][0] * 100.0
#print(x)

#reading the data from pickle
state_data_df = pd.read_pickle("state_data.pickle")

In [21]:
#function to get house price index data for entire USA as a benchmark
def Index_USA():
    df = q.get("FMAC/HPI_USA", authtoken=api_key)
    df["United States                  not seasonally adjusted"] = (df["United States                  not seasonally adjusted"]-df["United States                  not seasonally adjusted"][0]) / df["United States                  not seasonally adjusted"][0] * 100.0
    return df

In [29]:
#collecting the benchmark data
benchmark = Index_USA()


#Plotting of housing price index for all the states VS housing price index for entire USA (benchmark)
fig = plt.figure()
ax1 = plt.subplot2grid((1,1), (0,0))

    
state_data_df.plot(ax = ax1)
benchmark.plot(ax = ax1, color = "k", linewidth = 10)
plt.xlabel("Year")
plt.ylabel("Housing Price Index")
plt.title("Comparison of Housing Price Index\nall states\nVsentire USA")
plt.legend().remove()
plt.show()